## Setup
Installs and imports necessary modules from Github.

Trouble running this? Try restarting the Colab session ("Runtime" --> "Restart Session").

If you're working in your own branch in the Github repo, change the wget commands to point towards that branch.

In [3]:
# Get all scripts from the modules folder in Github
!mkdir -p modules
base_url = "https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/lukes_branch/modules/"
files = ["configs.py", "plots.py", "network.py", "tasks.py", "bci.py"]
for file in files:
    !wget -q "{base_url}{file}" -O "modules/{file}"

# Imports for this script
import numpy as np
from os.path import join
from modules.plots import TaskViz, ManifoldViz, ExperimentSummaryViz
import modules.tasks as tasks
import modules.configs as configs
import importlib

##Train and Evaluate the Model

In [6]:
importlib.reload(tasks)

# ====================
# == Initialization ==
# ====================

seeds = [1,2,3,4,5]
experiments = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
eigenval_list = []


# =====================
# == Run Experiments ==
# =====================

for seed in seeds:

  root_out_dir = f"./results/seed{seed}"

  for ntargets in experiments:


    # ======================
    # == Setup Experiment ==
    # ======================

    exp_name = f"{ntargets}Targets" # Name of this experiment
    exp_out_dir = join(root_out_dir, exp_name)

    cfg = configs.BasicExperimentConfig(ntrials=80, ntargets=ntargets, seed=seed)
    task =  tasks.GeometricReachingTask() # The task the RNN will learn
    rnn = cfg.rnn # Recurrent Neural Network
    bci = cfg.bci # Brain computer interface

    print(f"Starting experiment: {exp_name}")
    print(f"-- Random Seed: {seed}")
    print(f"-- Stimulus Shape: {task.stimuli.shape}")
    print(f"-- Targets Shape: {task.targets.shape}")


    # =======================
    # == Simulate Learning ==
    # =======================

    # -- Train the RNN
    rnn.relearn(cfg.ntrials, task.stimuli,
                task.stim_length, bci.decoder,
                cfg.feedback, task.targets)

    # -- Compute manifold
    manifold_out = rnn.calculate_manifold(
       task.stimuli,
       task.stim_length,
       cfg.ntrials_manifold
    )

    proj = manifold_out['xi2'] # Reshaped projection
    activity = manifold_out['activity_reshaped']
    order = manifold_out['order'] # Target indices for each trial
    evecs = manifold_out['evec'] # Eigenvectors

    # -- Train BCI
    bci.train(proj,task.targets[:,task.stim_length:,:],evecs,order)


    # ===========================================
    # == Visualize Results For This Experiment ==
    # ===========================================

    manifold_viz = ManifoldViz(manifold_out)
    task_viz = TaskViz(task, activity, bci, order, dt=rnn.dt)
    manifold_viz.save(join(exp_out_dir, "Manifold.html"), overwrite=True)
    task_viz.save(join(exp_out_dir, "Task.html"), overwrite=True)


    # ======================================
    # == Save Results For This Experiment ==
    # ======================================
    print("Saving results for experiment...")
    cfg.save(join(exp_out_dir, "result.pkl"), manifold_out, overwrite=True)
    print("Experiment complete!\n")


  # ==============================================
  # == Save Summary Results For All Experiments ==
  # ==============================================
  print("Saving summary results for all experiments...")
  summary_viz = ExperimentSummaryViz(experiments, eigenval_list)
  summary_viz.save(join(root_out_dir, "Summary.html"), overwrite=True)

Starting experiment: 1Targets
-- Random Seed: 1
-- Stimulus Shape: (6, 200, 6)
-- Targets Shape: (6, 200, 2)


  0%|          | 0/80 [00:00<?, ?it/s]


ValueError: shapes (800,1) and (6,) not aligned: 1 (dim 1) != 6 (dim 0)